# Assignment 5

## Part 1: File Structure

In [3]:
# import relevant libraries
import pandas as pd
import numpy as np
import re

from datetime import date

In [4]:
# import the data
raw_data = pd.read_csv("../data/atlantic.csv")

In [5]:
# make names in proper case
raw_data["Name"] = raw_data["Name"].str.strip().str.title()

# retain only landfall events
raw_data["Event"] = raw_data["Event"].str.strip()  # Remove spaces
raw_data = raw_data.query("Event == 'L'")  # Now filter

# create a compound key for later joining, and also distinguishes storm names used in multiple years
raw_data["Season and Name"] = raw_data["Date"].astype(str).str[:4] + " " + raw_data["Name"]

# retain only the columns I want# retain only the columns I want
selected_columns_table_1 = ["Season and Name", "ID", "Date", "Event", "Status", "Latitude", "Longitude", "Maximum Wind", "Minimum Pressure"]
raw_data = raw_data[selected_columns_table_1]

# save the cleaned data
file_name = "../data_clean/raw_data_cleaned"+ date.today().isoformat()+'.csv'
raw_data.to_csv(file_name)

## Part 2: Joins

In [7]:
## import another dataset

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_costliest_Atlantic_hurricanes"

# Read all tables from the Wikipedia page
tables = pd.read_html(url)

# Select the relevant table (2nd table about monetary damage is relevant)
dmg_table = tables[1]

# delete " †", which is used in the wikipedia table to denote "storm's impact in that season did not result in its name being retired"
dmg_table["Name"] = dmg_table["Name"].str.replace(" †", "", regex=False)

# create a compound key for joining
dmg_table["Season and Name"] = dmg_table["Season"].astype(str) + " " + dmg_table["Name"]

# retain only the columns I want
selected_columns_table_2 = ["Season and Name", "Nominal damage (Billions USD)", "Areas affected"]
dmg_table = dmg_table[selected_columns_table_2]

# Save the cleaned data
file_name = "../data_clean/damage_table_from_wikipedia"+ date.today().isoformat()+'.csv'
dmg_table.to_csv(file_name)


In [8]:
# join the tables together to make a big one
final_table = pd.merge(dmg_table, raw_data, how = 'inner', on = 'Season and Name')

final_table

,Season and Name,Nominal damage (Billions USD),Areas affected,ID,Date,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure
0,2005 Katrina,$125.0,Louisiana Mississippi The Bahamas United State...,AL122005,20050825,L,HU,26.0N,80.1W,70,984
1,2005 Katrina,$125.0,Louisiana Mississippi The Bahamas United State...,AL122005,20050829,L,HU,29.3N,89.6W,110,920
2,2005 Katrina,$125.0,Louisiana Mississippi The Bahamas United State...,AL122005,20050829,L,HU,30.2N,89.6W,105,928
3,2012 Sandy,$68.7,The Caribbean United States East Coast Eastern...,AL182012,20121024,L,HU,17.9N,76.6W,75,971
4,2012 Sandy,$68.7,The Caribbean United States East Coast Eastern...,AL182012,20121025,L,HU,20.0N,76.0W,100,954
...,...,...,...,...,...,...,...,...,...,...,...
121,2002 Lili,$1.2,Lesser Antilles Greater Antilles Gulf Coast of...,AL132002,20021001,L,HU,21.3N,83.0W,90,971
122,2002 Lili,$1.2,Lesser Antilles Greater Antilles Gulf Coast of...,AL132002,20021001,L,HU,22.1N,84.0W,90,971
123,2002 Lili,$1.2,Lesser Antilles Greater Antilles Gulf Coast of...,AL132002,20021003,L,HU,29.5N,92.2W,80,963
124,1994 Alberto,$1.0,Southeastern United States,AL011994,19940703,L,TS,30.4N,86.5W,55,993


In [9]:
# save the file
file_name = "../results/final_table"+ date.today().isoformat()+'.csv'
final_table.to_csv(file_name)

### Why this inner join?
The new table aims to analyse the economic impact of hurricanes that made landfall(s). We need the "raw_data" table to screen out only hurricanes that made landfall, and the "dmg_table" to list out the economic loss. Other joins will either result in the listing of storms that didn't make landfall, or storms that are lessly costly in terms of damage.

No NaN value is created as inner join will only select records that exist in both tables.